In [ ]:
import requests, json, glob, pandas as pd
from haversine import haversine

#information
n =800.0  # limit distance(milli)

#def1 kakao local
def get_address(lat, lng):
    url = "https://dapi.kakao.com/v2/local/geo/coord2address.json?x="+lng+"&y="+lat + "&input_coord=WGS84"
    headers = {"Authorization": "KfkaoAK aae1cae5b0e03ac354ffbaea7ce960de "}
    api_json = requests.get(url, headers=headers)
    full_address = json.loads(api_json.text)
    return full_address

#1 csv file load `경진대회용 모의데이터`
path = r"제2회 민원 데이터 분석경진대회용 불법주청차 모의 데이터/*.xlsx"
file_path = glob.glob(path)
data = pd.DataFrame()
for f in file_path:    
    tmepd = pd.read_excel(f)
    data =pd.concat([data,tmepd],ignore_index=True )

#2 json file `전국주차장정보표준데이터`
with open(r"제2회 민원 데이터 분석경진대회용 불법주청차 모의 데이터/전국주차장정보표준데이터.json", 'rb') as f:
    res =json.loads(f.read().decode("utf-8")) 

#3 parkdf pd `전국주차장정보표준데이터`
ppname=[]
for i in range(len(res["records"])):
    if (res["records"][i]["위도"] and res["records"][i]["경도"] != "") and (res["records"][i]["주차장구분"] == "공영"):
        if(res["records"][i]["소재지도로명주소"] != ""):
                address =res["records"][i]["소재지도로명주소"]
        else:
                address =res["records"][i]["소재지지번주소"]
        ppname.append([res["records"][i]["위도"],res["records"][i]["경도"],res["records"][i]["주차장명"] ,\
                    res["records"][i]["주차장유형"], address,res["records"][i]["주차구획수"], res["records"][i]["요금정보"] ,\
                    res["records"][i]["평일운영시작시각"], res["records"][i]["평일운영종료시각"], res["records"][i]["토요일운영시작시각"] ,\
                    res["records"][i]["토요일운영종료시각"], res["records"][i]["공휴일운영시작시각"], res["records"][i]["공휴일운영종료시각"]
                ])
parkdf =pd.DataFrame(ppname, columns=["P위도", "P경도", "주차장명", "주차장유형","주차장장소","주차구획수", "요금정보" ,\
                    "평일운영시작시각", "평일운영종료시각", "토요일운영시작시각", "토요일운영종료시각", "공휴일운영시작시각", "공휴일운영종료시각"
                     ])
parkdf =parkdf.drop_duplicates(['주차장명'])

#4 countdf pd `경진대회용 모의데이터` 
cdata =data.iloc[:,1:3].value_counts() #civil  complaint
countdf =pd.DataFrame(list(cdata.index),columns=["위도","경도"])
countdf["불법주정차개수"]=list(cdata.values)
#4-1 
# ccname=[]
# for i in range(len(countdf.index)):
#     full_address = get_address(str(countdf.iat[i,0]), str(countdf.iat[i,1]))
#     if (full_address['documents']): 
#         if full_address['documents'][0]['road_address'] != None: 
#             ccname.append(full_address['documents'][0]['road_address']['address_name'])
#         elif full_address['documents'][0]['address'] != None:
#             ccname.append(full_address['documents'][0]['address']['address_name'])
#     else:
#         ccname.append("")    
# countdf["불법주정차장소"] =ccname

#5 #distance haversine calculation
spa, coodtemp ={},{}
dis, id, size, sid, =[],[],[],[]
temp =0
for j in range(len(cdata.index)):    
    for i in range(len(parkdf.index)):
        if n  >= (haversine( (cdata.index[j][0],cdata.index[j][1]),(float(parkdf.iat[i,0]),float(parkdf.iat[i,1])), unit = 'm')):
            coodtemp[parkdf.iat[i,2]] = haversine( (cdata.index[j][0],cdata.index[j][1]),(float(parkdf.iat[i,0]),float(parkdf.iat[i,1])), unit = 'm') #distance haversine calculation
            spa[parkdf.iat[i,2]]  =int(parkdf.iat[i,5])
    if temp < len(coodtemp):
        temp =len(coodtemp)
    cood = dict(sorted(coodtemp.items(), key=lambda x: x[1]))
    id.append(list(cood))
    dis.append(list(cood.values()))
    sid.append([k for k,v in spa.items() if max(spa.values()) == v]) #id
    size.append([v for _,v in spa.items() if max(spa.values()) == v]) #크기
    coodtemp.clear()
    spa.clear()
    cood.clear()

#6-1 arounddf
for i in range(len(id)):
    if(len(id[i]) <temp):
        id[i].extend(["" for _ in range(temp-len(id[i]))])
        dis[i].extend(["" for _ in range(temp-len(dis[i]))])
arounddf =pd.DataFrame(list(cdata.index),columns=["위도","경도"])
iddf = pd.DataFrame(id, columns=["근처주차장명"+str(i) for i in range(1,temp+1)])
disdf=pd.DataFrame(dis, columns=["근처주차장거리"+str(i) for i in range(1,temp+1)])
arounddf =pd.concat([arounddf, iddf, disdf], axis=1)

#6-2 aroundspa
temp2=0
aroundspa =pd.DataFrame(list(cdata.index),columns=["위도","경도"])
for i in range(len(sid)):
    if temp2 < len(sid[i]):
        temp2 =len(sid[i])
for i in range(len(sid)):
    if((len(sid[i]) <temp2)):
        sid[i].extend(["" for _ in range(temp2-len(sid[i]))])
        size[i].extend(["" for _ in range(temp2-len(size[i]))])
siddf = pd.DataFrame(sid, columns=["근처최대주차장명"+str(i) for i in range(1,temp2+1)])
sizesdf =pd.DataFrame(size, columns=["근처최대주차구획수"+str(i) for i in range(1,temp2+1)])
aroundspa = pd.concat([aroundspa,siddf, sizesdf], axis=1)

#7 result save db
with pd.ExcelWriter('result.xlsx') as writer:
    data.to_excel(writer, sheet_name='data', index=False)
    parkdf.to_excel(writer, sheet_name='parkdf', index=False)
    countdf.to_excel(writer, sheet_name='countdf', index=False)
    arounddf.to_excel(writer, sheet_name='arounddf', index=False)
    aroundspa.to_excel(writer, sheet_name='aroundspa', index=False)